In [1]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

tokenizer = GPT2TokenizerFast.from_pretrained('gpt2-large')
model = GPT2LMHeadModel.from_pretrained('gpt2-large').cuda()

2023-03-02 10:36:26.415836: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 10:36:27.567126: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-02 10:36:27.567151: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-02 10:36:32.668796: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [89]:
tokenized = tokenizer.batch_encode_plus(['Barack Obama was the 44th president'], return_tensors='pt')
output = model(
    input_ids=tokenized['input_ids'].cuda(),
    attention_mask=tokenized['attention_mask'].cuda(),
    output_hidden_states=True)

In [90]:
tokenized = tokenizer('president', return_tensors='pt')['input_ids'].cuda()
president = model.transformer.wte(tokenized)[0]
import torch
for i, layer in enumerate(output.hidden_states):
    diff = -1
    if i > 0:
        # diff = torch.cosine_similarity(layer[:,-1,:], output.hidden_states[i - 1][:,-1,:], dim=1).item()
        diff = torch.cosine_similarity(layer[:,-1,:], model.transformer.wte.weight, dim=1)
        diff = tokenizer.decode(torch.topk(diff, 20).indices)
        # diff = torch.norm(layer[:,-1,:], dim=1).item()
    layer = model.lm_head(layer)
    print(i, tokenizer.decode(layer.argmax(dim=-1)[0][-1]), diff)

0 : -1
1 : : The In If This a Only It What Not " They All With A
 There ( Even When
2 30  If The $30 This20 In: Not Even When It
 Only What They A There That "
3  Less  If The This Not It Only They Even That
 What When $20 In There Less30: All
4  If  If The This They
 It In That A When What All There Only Not Even Since if $ We
5  Logged  The If This
 When It They There What A All In Only That 1 We Not Even2030
6  Logged  If The This
 When They A What20 It There30 In All We Not 1 Only Since40
7  Logged  The If This
 When A In We Only There What It 12030 They ( Since All measured
8  Logged  If This The When
 We Only They 1 In A There30 Since20 What It1 Not Because
9  Logged  If When This The 1
2030 Since1 Only

 In What$ $ They We There Because
10  Logged  If 1 When20 $30 The1 Logged (

$40 They
 Since-- This35 A
11  Logged  $ 13020$3540 When If1 The50
 Logged 20 (

-- 50 Currently
12  Logged  1 $$1 The
30 When 5050 If20 Logged A40 ( 20

35 Currently
13  Logged  The $ 1
 A1$ If 50 In Wh

In [82]:
model.transformer.wte.weight.shape

torch.Size([50257, 1280])